In [1]:
from pyspark.sql import SparkSession

# Initialize a Spark session (if not already done)
spark = SparkSession.builder \
    .appName("ModelTraining") \
    .getOrCreate()

# Define your S3 URI (replace with your bucket and path)
s3_path = "s3://aws-emr-studio-201107706822-us-east-1/1746053900403/e-6HYCE3WANA8IROUS7NBVDSOU4/processed/final_windows/final_6"



VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1747513138169_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
user_df = spark.read.parquet(s3_path)

# Show the top rows to confirm the data is loaded
user_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------+--------------------+----------------+--------------------+-----------------------+
| UserID|  JobID|          job_vector|WorkHistoryCount|TotalYearsExperience|user_job_history_vector|
+-------+-------+--------------------+----------------+--------------------+-----------------------+
|1001081| 561355|[-0.0106009230630...|             5.0|                 5.0|   [-0.1833965631163...|
|1001081| 456489|[-0.1183443906761...|             5.0|                 5.0|   [-0.1833965631163...|
|1001081| 283956|[-0.0301933131429...|             5.0|                 5.0|   [-0.1833965631163...|
|1001081| 507809|[-2.1248710496972...|             5.0|                 5.0|   [-0.1833965631163...|
|1001081| 871635|[-0.0759097041237...|             5.0|                 5.0|   [-0.1833965631163...|
|1001081| 975389|[0.03114965191237...|             5.0|                 5.0|   [-0.1833965631163...|
|1001081|1102813|[-0.2300305172801...|             5.0|                 5.0|   [-0.18339656

In [3]:
user_df = user_df.select("UserID", "job_vector")
user_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+
| UserID|          job_vector|
+-------+--------------------+
|1001081|[-0.0106009230630...|
|1001081|[-0.1183443906761...|
|1001081|[-0.0301933131429...|
|1001081|[-2.1248710496972...|
|1001081|[-0.0759097041237...|
|1001081|[0.03114965191237...|
|1001081|[-0.2300305172801...|
|1001081|[-0.0936919464709...|
|1001081|[-0.0663329315971...|
|1001081|[-0.1230443818494...|
|1001081|[-0.1392525945508...|
|1001081|[-0.2028739829381...|
|1001081|[-0.0871820227528...|
|1001081|[-0.0902447275475...|
|1001081|[-0.0613519765327...|
|1001522|[-0.0063639413043...|
|1005556|[0.00535736709094...|
|1005556|[-0.0350467577140...|
|1005556|[0.05736516155388...|
|1005598|[0.05728596615556...|
+-------+--------------------+
only showing top 20 rows

In [4]:
jobs_df = spark.read.parquet("s3://aws-emr-studio-201107706822-us-east-1/1746053900403/e-6HYCE3WANA8IROUS7NBVDSOU4/processed/windows/window=6/")
jobs_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------+--------------------+
|JobID|WindowID|          job_vector|
+-----+--------+--------------------+
|    6|       6|[0.07758777074199...|
|  177|       6|[0.07699037495889...|
|  179|       6|[-0.0493208222126...|
|  181|       6|[-0.1097340371105...|
|  184|       6|[-0.0565329056169...|
|  188|       6|[-0.1017359794081...|
|  190|       6|[0.08878653105865...|
|  192|       6|[-0.0108238010137...|
|  195|       6|[-0.0543442409005...|
|  196|       6|[-0.0235997953059...|
|  199|       6|[-0.0645179383553...|
|  202|       6|[-0.0752751027621...|
|  204|       6|[-0.0655673609137...|
|  205|       6|[0.10172618960496...|
|  206|       6|[-0.0871820227528...|
|  207|       6|[0.01857558905896...|
|  208|       6|[-0.0763380943265...|
|  209|       6|[-0.0477366117294...|
|  210|       6|[-0.1486951116099...|
|  211|       6|[-0.0112544465144...|
+-----+--------+--------------------+
only showing top 20 rows

In [5]:
from pyspark.ml.linalg import Vectors, DenseVector
from pyspark.sql.functions import udf, col, avg
from pyspark.sql.types import ArrayType, DoubleType

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Convert DenseVector to array for averaging
vector_to_array_udf = udf(lambda v: v.toArray().tolist(), ArrayType(DoubleType()))
df_array = user_df.withColumn("job_vector_array", vector_to_array_udf("job_vector"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
df_array.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+--------------------+
| UserID|          job_vector|    job_vector_array|
+-------+--------------------+--------------------+
|1001081|[-0.0106009230630...|[-0.0106009230630...|
|1001081|[-0.1183443906761...|[-0.1183443906761...|
|1001081|[-0.0301933131429...|[-0.0301933131429...|
|1001081|[-2.1248710496972...|[-2.1248710496972...|
|1001081|[-0.0759097041237...|[-0.0759097041237...|
|1001081|[0.03114965191237...|[0.03114965191237...|
|1001081|[-0.2300305172801...|[-0.2300305172801...|
|1001081|[-0.0936919464709...|[-0.0936919464709...|
|1001081|[-0.0663329315971...|[-0.0663329315971...|
|1001081|[-0.1230443818494...|[-0.1230443818494...|
|1001081|[-0.1392525945508...|[-0.1392525945508...|
|1001081|[-0.2028739829381...|[-0.2028739829381...|
|1001081|[-0.0871820227528...|[-0.0871820227528...|
|1001081|[-0.0902447275475...|[-0.0902447275475...|
|1001081|[-0.0613519765327...|[-0.0613519765327...|
|1001522|[-0.0063639413043...|[-0.0063639413043...|
|1005556|[0.

In [8]:
# Compute average for each vector dimension across all jobs per user
avg_exprs = [avg(col("job_vector_array")[i]).alias(f"avg_{i}") for i in range(100)]
user_avg_vectors = df_array.groupBy("UserID").agg(*avg_exprs)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
from pyspark.ml.linalg import Vectors, VectorUDT

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
def to_dense_vector(*vals):
    return Vectors.dense(vals)

to_vector_udf = udf(to_dense_vector, VectorUDT())

user_avg_vectors = user_avg_vectors.withColumn(
    "user_vector_avg", to_vector_udf(*[col(f"avg_{i}") for i in range(100)])
).select("UserID", "user_vector_avg")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
user_avg_vectors.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+
| UserID|     user_vector_avg|
+-------+--------------------+
|1010262|[0.10353731150728...|
|1104594|[0.02520169771388...|
|1178270|[0.01746556015362...|
|1181914|[0.07107934607740...|
|1192259|[0.11633230284503...|
|1276440|[0.05270350228205...|
| 131949|[0.02566017724968...|
|1342177|[0.01242057485684...|
|1363821|[0.02092127936583...|
|1367908|[0.04452441621672...|
|1380502|[0.01955687982034...|
|1464749|[0.04700561062283...|
| 194378|[5.78993730919642...|
| 231583|[0.02965426139888...|
| 249011|[0.08015021835690...|
| 264125|[-0.0927807517868...|
| 278220|[0.03807248266663...|
|  33174|[0.02504478503383...|
| 393343|[0.02477552513090...|
| 429742|[-0.0326636415665...|
+-------+--------------------+
only showing top 20 rows

In [13]:
applied_df = spark.read.option("header", True).option("sep", "\t").csv("s3://aws-emr-studio-201107706822-us-east-1/1746053900403/e-6HYCE3WANA8IROUS7NBVDSOU4/apps.tsv")
applied_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------+-----+--------------------+-------+
|UserID|WindowID|Split|     ApplicationDate|  JobID|
+------+--------+-----+--------------------+-------+
|    47|       1|Train|2012-04-04 15:56:...| 169528|
|    47|       1|Train|2012-04-06 01:03:...| 284009|
|    47|       1|Train|2012-04-05 02:40:...|   2121|
|    47|       1|Train|2012-04-05 02:37:...| 848187|
|    47|       1|Train|2012-04-05 22:44:...| 733748|
|    47|       1|Train|2012-04-05 02:34:...| 576958|
|    47|       1|Train|2012-04-05 22:55:...| 262470|
|    47|       1|Train|2012-04-05 02:38:...| 602298|
|    72|       1|Train|2012-04-02 22:36:...| 834662|
|    72|       1|Train|2012-04-07 15:19:...|1020903|
|    72|       1|Train|2012-04-07 17:38:...| 180313|
|    72|       1|Train|2012-04-30 20:05:...| 480634|
|    72|       1|Train|2012-04-20 02:51:...| 564184|
|    80|       1|Train|2012-04-04 10:53:...| 908909|
|   123|       1|Train|2012-04-02 20:57:...|1080147|
|   123|       1|Train|2012-04-02 21:03:...|10

In [14]:
applied_df = applied_df.filter(col("WindowID") == 6)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
jobs_vecs = spark.read.parquet("s3://aws-emr-studio-201107706822-us-east-1/1746053900403/e-6HYCE3WANA8IROUS7NBVDSOU4/processed/windows/window=6/")
jobs_vecs.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------+--------------------+
|JobID|WindowID|          job_vector|
+-----+--------+--------------------+
|    6|       6|[0.07758777074199...|
|  177|       6|[0.07699037495889...|
|  179|       6|[-0.0493208222126...|
|  181|       6|[-0.1097340371105...|
|  184|       6|[-0.0565329056169...|
|  188|       6|[-0.1017359794081...|
|  190|       6|[0.08878653105865...|
|  192|       6|[-0.0108238010137...|
|  195|       6|[-0.0543442409005...|
|  196|       6|[-0.0235997953059...|
|  199|       6|[-0.0645179383553...|
|  202|       6|[-0.0752751027621...|
|  204|       6|[-0.0655673609137...|
|  205|       6|[0.10172618960496...|
|  206|       6|[-0.0871820227528...|
|  207|       6|[0.01857558905896...|
|  208|       6|[-0.0763380943265...|
|  209|       6|[-0.0477366117294...|
|  210|       6|[-0.1486951116099...|
|  211|       6|[-0.0112544465144...|
+-----+--------+--------------------+
only showing top 20 rows

In [16]:
from pyspark.sql import SparkSession, Row
from pyspark.sql import functions as F

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
positive_df = applied_df.select("UserID", "JobID").distinct().withColumn("label", F.lit(1))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
positive_df.show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+-----+
|UserID|  JobID|label|
+------+-------+-----+
| 13922| 853364|    1|
| 14464| 263639|    1|
| 20572| 627008|    1|
| 30220| 210984|    1|
| 36870| 631929|    1|
| 40525| 928063|    1|
| 47944| 314946|    1|
| 59852| 483052|    1|
| 72256|1100586|    1|
| 73609|  78269|    1|
| 77113| 497053|    1|
| 82515| 194985|    1|
| 86447| 606659|    1|
| 87251|   1215|    1|
|122335| 205186|    1|
|130857| 311469|    1|
|167483| 882676|    1|
|172969| 407928|    1|
|180914| 896188|    1|
|185348| 601902|    1|
+------+-------+-----+
only showing top 20 rows

In [22]:
all_jobs = jobs_vecs.select("JobID").distinct()
users = applied_df.select("UserID").distinct()

# Cartesian product and filter out known applications
user_job_pairs = users.crossJoin(all_jobs)
negative_df = user_job_pairs.join(positive_df, on=["UserID", "JobID"], how="left_anti")  # only non-applied jobs

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
# Sample negatives per user
negatives_per_user = 3
neg_df = negative_df.sampleBy("UserID", fractions={uid: 1.0 for uid in users.rdd.map(lambda r: r.UserID).collect()}, seed=42).limit(positive_df.count() * negatives_per_user)

neg_df = neg_df.withColumn("label", F.lit(0))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
all_df = positive_df.union(neg_df)

# Join with user and job vectors
df = all_df \
    .join(user_avg_vectors, on="UserID", how="inner") \
    .join(jobs_vecs, on="JobID", how="inner")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------+-----+--------------------+--------+--------------------+
|  JobID| UserID|label|     user_vector_avg|WindowID|          job_vector|
+-------+-------+-----+--------------------+--------+--------------------+
| 431424| 851366|    1|[0.06143603974552...|       6|[0.15489870952084...|
|1031017|1193265|    1|[0.03149212335574...|       6|[0.05213007447185...|
| 212175|1036710|    1|[0.02426195174953...|       6|[0.06290548449382...|
|  90228|1343781|    1|[0.02658318885780...|       6|[0.04625847889110...|
|1097471|1418241|    1|[0.01848544198317...|       6|[0.03605543842746...|
| 431424| 742341|    1|[0.06691129614989...|       6|[0.15489870952084...|
| 794130| 268352|    1|[0.07794334373064...|       6|[0.00121404060960...|
| 346809| 588350|    1|[0.00124170863894...|       6|[0.05023472688536...|
| 431424|1452190|    1|[0.03015024216213...|       6|[0.15489870952084...|
| 674313| 943315|    1|[0.02191119514872...|       6|[0.00595230846242...|
| 282680|1045481|    1|[0

In [37]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, DoubleType
from pyspark.ml.linalg import VectorUDT

def vector_to_list(v):
    return v.toArray().tolist()

vector_to_list_udf = udf(vector_to_list, ArrayType(DoubleType()))

df_1 = df.withColumn("user_vec_array", vector_to_list_udf("user_vector_avg"))
df_1 = df_1.withColumn("job_vec_array", vector_to_list_udf("job_vector"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
df_1.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+-----+--------------------+--------+--------------------+--------------------+--------------------+
| JobID| UserID|label|     user_vector_avg|WindowID|          job_vector|      user_vec_array|       job_vec_array|
+------+-------+-----+--------------------+--------+--------------------+--------------------+--------------------+
| 28024|1466633|    1|[0.02622618472881...|       6|[-0.0159825073893...|[0.02622618472881...|[-0.0159825073893...|
|282680|1062211|    1|[-0.0259603706615...|       6|[-0.0773326169699...|[-0.0259603706615...|[-0.0773326169699...|
|156645| 110546|    1|[-0.0015317823352...|       6|[-0.0081193856331...|[-0.0015317823352...|[-0.0081193856331...|
|212175| 301128|    1|[0.03891433157345...|       6|[0.06290548449382...|[0.03891433157345...|[0.06290548449382...|
| 28024| 444893|    1|[-8.7489770879573...|       6|[-0.0159825073893...|[-8.7489770879573...|[-0.0159825073893...|
| 28024| 746330|    1|[0.02447650985592...|       6|[-0.0159825073893...

In [39]:
user_vec_dim = df_1.select(F.size("user_vec_array")).first()[0]
job_vec_dim = df_1.select(F.size("job_vec_array")).first()[0]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
user_vec_dim

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

100

In [41]:
job_vec_dim

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

100

In [45]:
df_1.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------+-----+--------------------+--------+--------------------+--------------------+--------------------+
|  JobID| UserID|label|     user_vector_avg|WindowID|          job_vector|      user_vec_array|       job_vec_array|
+-------+-------+-----+--------------------+--------+--------------------+--------------------+--------------------+
| 431424| 851366|    1|[0.06143603974552...|       6|[0.15489870952084...|[0.06143603974552...|[0.15489870952084...|
|1031017|1193265|    1|[0.03149212335574...|       6|[0.05213007447185...|[0.03149212335574...|[0.05213007447185...|
| 212175|1036710|    1|[0.02426195174953...|       6|[0.06290548449382...|[0.02426195174953...|[0.06290548449382...|
|  90228|1343781|    1|[0.02658318885780...|       6|[0.04625847889110...|[0.02658318885780...|[0.04625847889110...|
|1097471|1418241|    1|[0.01848544198317...|       6|[0.03605543842746...|[0.01848544198317...|[0.03605543842746...|
| 431424| 742341|    1|[0.06691129614989...|       6|[0.15489870

In [46]:
for i in range(100):  # adjust based on actual vector size
    df_1 = df_1.withColumn(f"user_{i}", F.col("user_vec_array")[i])
    df_1 = df_1.withColumn(f"job_{i}", F.col("job_vec_array")[i])

df_1 = df_1.drop("user_vector_avg", "job_vector")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
df_1.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------+-----+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----

In [48]:
df_1.write.mode("overwrite").parquet("s3://aws-emr-studio-201107706822-us-east-1/1746053900403/e-6HYCE3WANA8IROUS7NBVDSOU4/processed/training_data_6/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df_1.filter(F.col("label") == 0).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…